In [27]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.preprocessing import normalize
import multiprocessing
from eunjeon import Mecab
import eunjeon
import konlpy
import textlib as tl
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
MAX_VOCAB_SIZE = 10000
EMBEDDING_SIZE = 100
WORKERS = multiprocessing.cpu_count() - 1
WINDOW_SIZE = 5
EPOCHS = 10

UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
tokenizer = Mecab()

In [4]:
def make_w2v_model(in_file_name, out_file_name,
                   max_vocab_size=10000, embedding_size=100,
                   epochs=10, window=5, workers=3):
    # 빈도수 상위 vocab_size 내에 존재하는 단어 중 최소 빈도수를 구함
    def get_min_freq_count(sentences, max_freq_rank):
        from itertools import chain
        import nltk

        fdist = nltk.FreqDist(chain.from_iterable(sentences))
        return fdist.most_common(max_freq_rank)[-1][1] # the count of the the top-kth word

    # 단어 모음
    corpus = [sentence.strip().split(' ') 
              for sentence in open(in_file_name, 'r', encoding='utf-8').readlines()]
    # 빈도수 상위 n위의 최소빈도수 구함 (word2vec 훈련 시 그 이하는 버리기 위함)
    min_freq_cnt = get_min_freq_count(corpus, max_vocab_size)
    print(f'{max_vocab_size}개의 단어 내에서 최소 빈도수는 {min_freq_cnt}입니다.')
    
    # gensim word2vec call
    w2v_model = Word2Vec(corpus, 
                     size=embedding_size, 
                     workers=workers, 
                     min_count=min_freq_cnt,
                     sg=1, 
                     iter=epochs,
                     window=window)
    # 저장
    w2v_model.save(out_file_name)        
    
    return _post_w2v_process(w2v_model)

In [5]:
def _post_w2v_process(w2v_model):    
    # unknown, padding 토큰 추가
    def _append_unk_pad_vectors(embeddings):
        embedding_size = embeddings.shape[1]
        def get_truncated_normal(mean=0, sd=1, low=-1, upp=1):
            from scipy.stats import truncnorm
            return truncnorm(
                    (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

        return np.append(embeddings, 
                         get_truncated_normal().rvs(embedding_size * 2).reshape(2, embedding_size), axis=0)    
        
    index2word = w2v_model.wv.index2word
    # unk, pad 추가
    index2word.append( UNK_TOKEN )
    index2word.append( PAD_TOKEN )    
    
    w2v = w2v_model.wv.vectors
    # unk, pad에 해당하는 normal 초기화된 벡터 추가
    w2v = _append_unk_pad_vectors(w2v)
    # cosine유사도 체크를 위해 normalize
    unit_w2v = normalize(w2v, norm='l2', axis=1)

    # word를 index로 변환
    word2index = {w:i for i, w in enumerate(index2word)}
    # 사전. word를 vector로 변환
    dictionary = {w:v for w, v in zip(index2word, unit_w2v)}    
    
    return {
        'w2v_model'   : w2v_model, 
        'index2word'  : index2word, 
        'word2index'  : word2index,

        'weight'      : w2v,
        'norm_weight' : unit_w2v
    }

In [7]:
corpora_file_name = 'D:/data/telco_corpora.dat'
w2v_model_file_name = f'd:/data/telco_w2v_{MAX_VOCAB_SIZE}_{EMBEDDING_SIZE}_{EPOCHS}_{WINDOW_SIZE}.dat'


In [ ]:

# word2vec 모델 만들고 저장. 그 외 필요한 값들 리턴
w2v_output = make_w2v_model(corpora_file_name, 
                   w2v_model_file_name, 
                   max_vocab_size=MAX_VOCAB_SIZE, 
                   embedding_size=EMBEDDING_SIZE,
                   epochs=EPOCHS,
                   window=WINDOW_SIZE,
                   workers=WORKERS)



In [77]:
print( len(w2v_output['index2word']) )
print( len(w2v_output['word2index']) )
print( len(w2v_output['weight']) )

print( w2v_output['index2word'][200] )
print( w2v_output['word2index']['약정'] )
print( w2v_output['weight'][2583] )
print( w2v_output['norm_weight'][2583] )

10200
10200
10200
수납
61
[-0.35776106  0.21665499  0.12065426 -0.07808045 -0.08555077 -0.31346983
  0.1906666   0.69993132 -0.27874622  0.33214924 -0.10627281 -0.43345928
  0.64351821 -0.40301988 -0.10826819 -0.57930028  0.01342783 -0.02467114
 -0.00303033 -0.24342476  0.12172619 -0.22651088 -0.29879901  0.25398812
 -0.26312226  0.08500879 -0.76019847  0.23448333  0.18769121 -0.24128641
 -0.58256441  0.47644502 -0.20451702 -0.28703746 -0.4114756   0.25096068
 -0.45316589  0.57999337  0.12603375  0.18206236  0.46993253 -0.36381519
 -0.42706221 -0.14927141  0.35819951  0.60059774  0.48026413 -0.08949742
 -0.37397689 -0.1537744  -0.45392779  0.6441046   0.171067   -0.21600319
 -0.29178634  0.24486649 -0.94137931  0.14310238  0.06903156  0.15202139
 -0.04857434 -0.44160995  0.31435215 -0.14665219 -0.49364892  0.4428457
 -0.63872004 -0.76166147  0.13702716  0.20451474 -0.83049208  0.00682165
  0.07786661 -0.4688977   0.11341438 -0.26523516  0.435202    0.15378253
 -0.21403897 -0.15495023 -0.

In [10]:
def plot_hist(hist):
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(1,2)
    loss_ax = axs[0,0]
    acc_ax = axs[0,1]

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper right')

    plt.show()

In [8]:
# 훈련 완료된 모델 있으면 로드해서 쓴다.
def load_w2v_model(model_file_name):
    w2v_model = Word2Vec.load(model_file_name)

    return _post_w2v_process(w2v_model)

w2v_output = load_w2v_model(w2v_model_file_name)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Dense, Embedding, Activation, LSTM, Bidirectional, GRU, Softmax, Dropout
from keras.preprocessing.sequence import pad_sequences

In [46]:
input_file_name = 'd:/data/SOR/sor_dataset.xlsx'
try:
    df = pd.read_excel(input_file_name, sheet_name=0, engine='openpyxl')
except FileNotFoundError:
    print(f'{input_file_name}이 없습니다! skip!')

# null 인 row가 하나라도 있으면 삭제
df.dropna(axis=0, inplace=True)


In [47]:
df.shape

(92483, 5)

In [48]:

# co(요청회사)가 SKT, SKB인 것만 추출
df = df[ ((df['co'] == 'SKT') | (df['co'] == 'SKB')) & \
             (
                (df['label'] != 'Configuration') & 
                (df['label'] != 'EAI/EIGW') &
                (df['label'] != 'I/F 유틸') &
                (df['label'] != 'MTOKTOK') &
                (df['label'] != 'PPS 상품권') &
                (df['label'] != 'Utility') &
                (df['label'] != '고객상담') &
                (df['label'] != '접근 관리') &
                (df['label'] != '코드 관리')
             )
       ]

In [49]:
df.shape

(92197, 5)

In [50]:
df.groupby('label').count()

,req_ym,co,req_br,sentence
label,,,,
Billing/OSS,6699,6699,6699,6699
CC,6484,6484,6484,6484
CRM,24176,24176,24176,24176
CTC,1676,1676,1676,1676
Customer Care,11064,11064,11064,11064
ERP,5661,5661,5661,5661
OCEAN,35,35,35,35
OSS,5458,5458,5458,5458
PRM,2597,2597,2597,2597


In [51]:
# 첫 모델은 sentence와 label만 써보자
df_zip = df[ ['sentence', 'label'] ]

y = df_zip.pop('label')
X = df_zip.pop('sentence')

In [52]:
# 문자열로 되어 있는 label을 categorical value로 변환
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [91]:
# 전처리 테스트
print( X[0] )
print()
cleansed_text = tl.clean_text( X[0] )
print( cleansed_text )
print()
tokenized_sentence = tl.segment_sentences(cleansed_text)
print( tokenized_sentence )
print()
corpora = ' '.join(tl.get_corpora(tokenized_sentence))
print(corpora)
sequence = [w2v_output['word2index'][t] if t in w2v_output['word2index'] 
                                        else w2v_output['word2index'][ UNK_TOKEN ]
                                        for t in corpora]

print(sequence)

소속영업장 변경 요청(SKB사내유치본점 -＞ 엘에스통신-채널고객팀) . SR-1705-0886;- 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 ;변경이 어렵습니다.;- 유통지원센터에서 청약 등록 시, 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요청 함 . 요청유형:자료수정,요청유형상세:PRM,검토/승인자성명:홍도희,검토/승인자사번:1700,검토승인자기간:2017-05-31,요청내용:1. 변경 전 유통망 : SKB사내유치본점2. 변경 후 유통망 : 엘에스통신(E00901)3. 서비스번호 : 7276564018 또는 1670-84914. 고객 : 세종화재해상자동차손해사정(주)5. 담당 AM : 박성M6. 적용 시점 : 6월 지급 분부터 적용(5월 영업에 대한)

소속영업장 변경 요청 SKB사내유치본점 엘에스통신 채널고객팀 . SR 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 변경이 어렵습니다. 유통지원센터에서 청약 등록 시 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요청 함 . 요청유형 자료수정 요청유형상세 PRM 검토/승인자성명 홍도희 검토/승인자사번 검토승인자기간 요청내용 . 변경 전 유통망 SKB사내유치본점 . 변경 후 유통망 엘에스통신 E . 서비스번호 또는 . 고객 세종화재해상자동차손해사정 주 . 담당 AM 박성M . 적용 시점 월 지급 분부터 적용 월 영업에 대한 

['소속영업장 변경 요청 SKB사내유치본점 엘에스통신 채널고객팀', 'SR 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 변경이 어렵습니다', '유통지원센터에서 청약 등록 시 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요청 함', '요청유형 자료수정 요청유형상세 PRM 검토/승인자성명 홍도희 검토/승인자사번 검토승인자기간 요청내용', '변

In [92]:
# 문장 전처리
preprocessed_X = []
print(f'{len(X)} 개의 데이터 존재 확인!')
for i, text in enumerate(X):
    try:
        # 클렌징
        cleansed_text = tl.clean_text(text)
    except TypeError:
        print(f'      {i+1} 번째 데이터에 문제가 있어 skip!')
        continue

    # 문장으로 분리하여 배열로 리턴
    sentences = tl.segment_sentences(cleansed_text)
    # 문장 배열을 입릭으로 받아 다시 하나의 문자열로 변환하여 X에 저장
    tokenized_sentence = tl.get_corpora(sentences)
    sequence = [w2v_output['word2index'][t] if t in w2v_output['word2index'] 
                                            else w2v_output['word2index'][ UNK_TOKEN ]
                                            for t in tokenized_sentence]
    preprocessed_X.append(sequence)

    if i % 5000 == 0 and i > 0:
        print(f'      {i} 번째 데이터 처리 완료!')

preprocessed_X = pad_sequences( preprocessed_X, maxlen=50, padding='post', value=w2v_output['word2index'][PAD_TOKEN] )

92197 개의 데이터 존재 확인!


KeyboardInterrupt: 

In [55]:
X_train, X_test, y_train, y_test = \
    train_test_split(preprocessed_X, y, test_size=0.3, random_state=42, stratify=y)

In [75]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test  = np.array(X_test)
y_test  = np.array(y_test)

print( len(X_train) )
print( len(X_test) )
print( len(y_train) )
print( len(y_test) )

64537
27660
64537
27660


32

In [61]:
vocab_size, embedding_dim = w2v_output['weight'].shape

In [62]:
class Attention(Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        # (batch, )
        self.w1 = Dense(units, activation='tanh')
        self.w2 = Dense(1)    
        self.softmax_ = Softmax(axis=1)
        
    def call(self, x):
        # (batch, seq, embedding_dim*2) -> # (batch, seq, dec_units)
        x = self.w1(x)
        # (batch, seq, dec_units) -> # (batch, seq, 1)
        score = self.softmax_( self.w2(x) )

        return tf.squeeze( tf.matmul(tf.transpose(x, perm=[0, 2, 1]), score ), axis=-1 )

In [68]:
class MoviePosNegClassifier(Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, 
                 batch_size, embedding_weights, apply_attention, train_embedding_layer, dropout, classes):
        super(MoviePosNegClassifier, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            weights=[embedding_weights])
        self.lstm = Bidirectional( LSTM(self.dec_units, return_sequences=True) )
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(classes, activation='softmax')
        self.do1 = Dropout(dropout)
        self.do2 = Dropout(dropout)
        
        self.attention = Attention(self.dec_units)
        self.embedding.trainable = train_embedding_layer
        self.apply_attention = apply_attention
        
    def call(self, x):
        # (batch, seq) -> (batch, seq, embedding_dim)        
        x = self.embedding(x)
        x = self.do1(x)
        # (batch, seq, embedding_dim) -> (batch, seq, embedding_dim*2)        
        x = self.lstm(x)
        
        # (batch, seq, embedding_dim*2) -> (batch, embedding_dim)        
        if self.apply_attention:
            x = self.attention(x)
        
        # (batch, embedding_dim) -> (batch, 128)    
        x = self.fc1(x)
        x = self.do2(x)
        x = self.fc2(x)
        
        return x

In [80]:
def train_and_evaluate(vocab_size, embedding_dim, dec_units, epochs, batch_size, 
       weights, apply_attention, train_embedding_layer, dropout, classes):
    model = MoviePosNegClassifier(
        vocab_size, 
        embedding_dim, 
        dec_units, 
        batch_size,
        weights,
        apply_attention,
        train_embedding_layer,
        dropout,
        classes
    )

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    
    test_score = model.evaluate(X_test, y_test, verbose=2)
    
    plot_hist(history)
    
    return f'{apply_attention}-{train_embedding_layer}-{dropout}', history, test_score

In [82]:
hyper_params = {
    'apply_attention': [True, False],
    'train_embedding_layer': [True, False],
    'dropout': [0.3],
    'weights': [w2v_output['weight'], w2v_output['norm_weight']]
}

histories = dict()
test_scores = dict()

for a in hyper_params['apply_attention']:
    for ef in hyper_params['train_embedding_layer']:
        for do in hyper_params['dropout']:
            for i, w in enumerate(hyper_params['weights']):
                model_name, history, test_score = \
                    train_and_evaluate(vocab_size, embedding_dim, 128, 10, 128, w, a, ef, do, len(label_encoder.classes_))
                model_name = f'{model_name}-{i}'
                histories[model_name] = history
                test_scores[model_name] = test_score

Epoch 1/10


UnimplementedError:  Cast string to int32 is not supported
	 [[node movie_pos_neg_classifier_2/embedding_2/Cast (defined at <ipython-input-68-36efa8b1f22a>:23) ]] [Op:__inference_train_function_10352]

Errors may have originated from an input operation.
Input Source operations connected to node movie_pos_neg_classifier_2/embedding_2/Cast:
 ExpandDims (defined at <ipython-input-80-a1029fdb7fb5>:20)

Function call stack:
train_function
